# THERMO_PROPERTY

## Overview
The `THERMO_PROPERTY` function retrieves a wide range of thermodynamic and transport properties (e.g., viscosity, enthalpy, entropy, density, heat capacity, etc.) for pure chemicals or mixtures at specified conditions using the [CalebBell/thermo](https://github.com/CalebBell/thermo) Python package. This enables engineers and scientists to perform advanced thermodynamic analyses directly in Excel. The function supports both SI and common engineering units, robust error handling, and can be used for both pure substances and mixtures.

The function works by creating a `Chemical` or `Mixture` object from the `thermo` package, depending on the input, and then retrieving the requested property using the appropriate attribute or method. For mixtures, the composition should be provided as a list of mole fractions.

## Usage
To use the function in Excel, enter the formula as follows (optional arguments in square brackets):

```excel
=THERMO_PROPERTY(property_name, identifier, temperature, pressure, [composition], [phase], [unit])
```

- `property_name`: Name of the property to retrieve (e.g., "viscosity", "enthalpy", "density").
- `identifier`: Chemical name or CAS number (string), or a list of names/CAS for mixtures.
- `temperature`: Temperature in K.
- `pressure`: Pressure in Pa.
- `composition`: (Optional) List of mole fractions for mixtures (e.g., [0.5, 0.5]).
- `phase`: (Optional) Phase to target ("l", "g", "s").
- `unit`: (Optional) Desired output unit (e.g., "kg/m^3", "J/mol").

## Arguments
| Argument      | Type         | Required | Description                                                      | Example         |
|:--------------|:-------------|:---------|:-----------------------------------------------------------------|:---------------|
| property_name | string       | Yes      | Name of property to retrieve                                     | "viscosity"   |
| identifier    | string/list  | Yes      | Chemical name/CAS (string) or list for mixtures                  | "water" or ["water", "ethanol"] |
| temperature   | float        | Yes      | Temperature in K                                                 | 298.15         |
| pressure      | float        | Yes      | Pressure in Pa                                                   | 101325         |
| composition   | list[float]  | No       | Mole fractions for mixtures                                      | [0.5, 0.5]     |
| phase         | string       | No       | Phase: "l" (liquid), "g" (gas), "s" (solid)                | "l"            |
| unit          | string       | No       | Desired output unit                                              | "kg/m^3"      |

## Returns
| Returns | Type   | Description                                 | Example     |
|:--------|:-------|:--------------------------------------------|:------------|
| value   | float  | Value of the requested property              | 0.00089     |
| error   | string | Error message if calculation fails           | "Error: ..." |

## Examples
### Retrieve Water Viscosity
**Input:**
| property_name | identifier | temperature | pressure |
|---------------|------------|-------------|----------|
| "viscosity"   | "water"    | 298.15      | 101325   |

**Call:**
```excel
=THERMO_PROPERTY("viscosity", "water", 298.15, 101325)
```
**Output:**
| value   |
    |---------|
    | 0.00089 |

### Mixture Density at 350 K
**Input:**
| property_name | identifier                | temperature | pressure | composition   |
|---------------|--------------------------|-------------|----------|--------------|
| "density"     | ["water", "ethanol"] | 350         | 101325   | [0.7, 0.3]   |

**Call:**
```excel
=THERMO_PROPERTY("density", ["water", "ethanol"], 350, 101325, [0.7, 0.3])
```
**Output:**
| value   |
    |---------|
    | 870.2   |

In [ ]:
import micropip
await micropip.install('chemicals')
from chemicals.viscosity import mu_IAPWS
from chemicals.iapws import iapws95_properties

PROPERTY_MAP = {
    'viscosity': lambda identifier, T, P: _get_viscosity(identifier, T, P),
    'heat_capacity': lambda identifier, T, P: _get_heat_capacity(identifier, T, P),
    'vapor_pressure': lambda identifier, T, P: _get_vapor_pressure(identifier, T, P),
}

def _get_viscosity(identifier, T, P):
    # Only water supported for now
    if identifier.lower() == 'water' or identifier == '7732-18-5':
        rho, *_ = iapws95_properties(T, P)
        return mu_IAPWS(T, rho)
    return None

def _get_heat_capacity(identifier, T, P):
    # Not implemented: would require mapping to correct function and coefficients
    return None

def _get_vapor_pressure(identifier, T, P):
    # Not implemented: would require mapping to correct function and coefficients
    return None

def thermo_property(property_name, identifier, temperature, pressure, composition=None, phase=None, unit=None):
    """
    Retrieve a thermodynamic or transport property for a chemical (pure only).
    Args:
        property_name (str): Name of property to retrieve.
        identifier (str): Chemical name or CAS (pure only).
        temperature (float): Temperature in K.
        pressure (float): Pressure in Pa.
        composition (list, optional): Not supported.
        phase (str, optional): Not supported.
        unit (str, optional): Not supported.
    Returns:
        float: Value of the requested property.
        str: Error message if calculation fails.
    """
    try:
        if isinstance(identifier, list):
            return None, 'Mixtures are not supported with the chemicals package.'
        if property_name not in PROPERTY_MAP:
            return None, f'Property {property_name} not supported.'
        value = PROPERTY_MAP[property_name](identifier, temperature, pressure)
        if value is None:
            return None, f'Property {property_name} not available for {identifier}.'
        return value, ''
    except Exception as e:
        return None, f'Error: {str(e)}'

In [ ]:
import gradio as gr

def gr_thermo_property(property_name, identifier, temperature, pressure, composition, phase, unit):
    value, error = thermo_property(property_name, identifier, temperature, pressure, composition, phase, unit)
    return value, error

demo = gr.Interface(
    fn=gr_thermo_property,
    inputs=[
        gr.Textbox(label="Property Name", value="viscosity"),
        gr.Textbox(label="Identifier (name or CAS, or list)", value="water"),
        gr.Number(label="Temperature (K)", value=298.15),
        gr.Number(label="Pressure (Pa)", value=101325),
        gr.Textbox(label="Composition (not supported)", value=""),
        gr.Textbox(label="Phase (not supported)", value=""),
        gr.Textbox(label="Unit (not supported)", value="")
    ],
    outputs=[
        gr.Number(label="Value"),
        gr.Textbox(label="Error")
    ],
    examples=[
        ["viscosity", "water", 298.15, 101325, "", "", ""],
    ],
    description="Retrieve thermodynamic/transport properties for pure chemicals using the CalebBell/chemicals package. Only a few properties for water are supported in this demo.",
    flagging_mode="never"
)
demo.launch()